# Enhancing GFM derived flood maps with Digital Terrain Models
We use the FLEXTH algorithm developed by the Joint Research Centre of the European Commission together with the FABDEM, a Digital Terrain Model, at a spatial resolution of 30m to enhance flood maps from the Global Flood Monitoring Database. This notebook features the code for the recommended practice by UN-SPIDER, accessible here.

The script is devided into four sections:
1. Loading necessary libraries.
2. Specifing user specific input and output directories.
3. Selecting the AOI and DTM source.
4. Mosaicing and Reprojecting GFM outputs.
5. FLEXTH

# 1. Loading the Libraries
This cell should be tested before the flood, after setting up your environment "flexth_env" with miniforge3 and mamba. See therefore the documentation in the recommended practice. If it can be executed without any error messages, you are prepared for deriving enhanced flood maps, in case of disaster.

In [2]:
import numpy as np
from osgeo import gdal
import glob, os
import logging
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.plot import show
from pathlib import Path
import cv2
from astropy.convolution import convolve
import warnings
import time
from scipy.spatial import cKDTree
import geopandas as gpd 
from shapely.geometry import box 
from matplotlib import pyplot

gdal.UseExceptions()

print('Libraries loaded')

Libraries loaded


# 2. Specifing input and output directories
The input directory is the downloaded and unziped! Folder from GFM. It makes sence to store it in a designated directory and not in your Downloads Folder. GFM has a naming convention for their downloads, which is nameofyouraoi_YYYY-MM-DDT00_hh_mm_YYYY_MM_DDTHH_MM_SS_....
It makes sence to store the output folder in the same directory. Please create it before running the script.

In [ ]:
# Unziped Download Folder from GFM:
input_dir = Path(r'C:/UrbanFloods/SriLanka/srilanka_2024-10-14T00_25_26_2024_11_20T13_39_01_033640/') 
output_dir = Path(r'C:/UrbanFloods/SriLanka/output/')

# 3. Selecting AOI and DTM sources
## AOI
You can specify an AOI in form of a geopackage (.gpkg) or a shapefile (.shp) in order not to run FLEXTH for the entire Sentinel-1 scene. The inputs and outputs will then be cropped to your AOI. If you don't have an AOI please write instead of the path to your roi, None.

## DTM
Other than for the AOI, without a DTM, the code won't work. Specify the path to your dtm in the variable dtm_path. It should be a .tif, for example the FABDEM, which we downloaded with the Google Earth Engine during the first steps of this practice.

In [ ]:
dtm_path = None # .tif
roi_path = None #.gpkg or .shp
dst_crs  = None # if 'None' rasters will be reprojected into EPSG 3857

# 4. Setting further Parameters
## Likelihood vs. Flood Layer
GFM also provides a likelihood layer, in which pixels are assigned a likelihood of flooding. Detailed documentation on how these values are calculated is provided here.
This is useful in case there are not enough pixels classified as flooded in the Observed Flood Extent Layer. In that case this layer is thresholded with a likelihood threshold (likelihood_thr). Default is 40%, which means pixels with a likelihood > 40% are considered as flooded. 

## Tiling
If your AOI is big, or the computational capacity of your machine is low, it is recommended to tile the inputs and outputs. This is for example relevant if you don't specify an AOI. The size of these tiles in number of pixels is set in param_tile_size. 

## Hydrological Parameters
A detailed explanation of the following parameters is given in the publication by Betterle and Salamon (2024) https://doi.org/10.5194/nhess-24-2817-2024. Brief explanations are also given in the recommended practice. Testing showed, that the default values are effective and robust in a wide range of settings. Nonetheless, parameters can be tweaked to mach specific needs and/or use cases. If the spatial resolution of the flood map is much larger/smaller than 10m, parameters may require adjustments.


In [ ]:
likelihood = True # True, if likelihood layer should be used as flood layer
likelihood_threshold = 40 # in Percent

if likelihood == True:
    print('Likelihood Layer will be thresholded with ' + str(likelihood_threshold) + '%.')

param_tiling      = False    # True to run FLEXTH on the tiled inputs
param_tile_inputs = True     # True to tile the inputs. If input is already tiled, select False. Relevant if "param_tiling = True"
param_tile_size   = 20000    # size of the squared tiles (pixels)


# Water level estimation method (options: 'method_A', 'method_B'): 
param_WL_estimation_method = 'method_A'  

# Select output: Water depth ("WD") , Water level ("WL"), both ("WL_WD")
param_output_map = "WL_WD"

# Parameters
param_threshold_slope           =  0.1     # S_max : border pixels steeper than this (D_z/D_x) are not used to estimate water level
param_size_gaps_close           =  0.01    # A_g   : up to this size (in km2) gaps in the flood map will be closed

param_border_percentile         =  0.5     # P: assign water level based on the percentile P of the elevation of border cells (valid if Method B is selected)
param_max_number_neighbors      =  100     # N_max: number of border pixels used to compute water level at a single location
param_min_flood_size            =  10      # N_min: if the number of valid pixels along the border is less than this, WL is estimated based on the distribution of the elevation of the pixels inside the flooded area
param_inner_quantile            =  0.98    # P*: for  flooded areas that don't meet the criteria above, uses this percentile of the elevation of the pixels inside the flooded area to estimate water levels
param_inverse_dist_exp          =  1       # alpha: inverse distance weighting exponent used to interpolate WL inside flooded areas 


param_max_propagation_distance  =  10    # D_max: maximum propagation distance in km
param_distance_range            =  10    # A_1/2: flooded areas of this size (km2) reach half of the maximum propagation distance  

param_WD_star                   =  10    # WD*: dummy water depth in cm assigned if estimated WL<DTM in initially delineated flooded areas


# 5. Mosaicing and Reprojecting GFM outputs
GFM outputs flood, permanent water body layers and exclusion masks in tiles and in latitude longitude coordinates. To clip the outputs to the AOI, and to run FLEXTH, the layers have to be one tif each, and need to be in a projected coordinate system. Therefore, all the layers are first mosaiced, then the flood layer is cropped and reprojected. The other layers and the dtm are then projected into the same grid.

You need to run this cell only once. If you change something concerning the parameters, you do not need to run this again. But if your AOI changes, please run it again. Therefore, you need to delete the older datasets first, as the script does not overwrite them automatically.

In [ ]:
def mosaic_tiles(input_dir, pattern):
    id = pattern.split('_')[1].split('*')[0]
    q = os.path.join(input_dir, pattern)
    flood_rst = glob.glob(q)
    
    rst_lst = []
    
    for fp in flood_rst:
        src = rasterio.open(fp)
        rst_lst.append(src)
        
    mosaic, out_trans = merge(rst_lst)
    
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                     "transform": out_trans,
                     "crs": src.crs
                     }
                  )
    
    with rasterio.open(input_dir / id +'_mosaic.tif', "w", **out_meta) as dest:
        dest.write(mosaic)

# File Path Patterns
lyr_ids = ['*ENSEMBLE_UNCERTAINTY*.tif', '*ENSEMBLE_EXCLAYER*.tif', '*ENSEMBLE_OBSWATER*.tif', '*ENSEMBLE_FLOOD*.tif']

# Mosaicing
for i in range(len(lyr_ids)):
    mosaic_tiles(input_dir, lyr_ids[i])


#%% Reprojections

# check if mosaics are already in the folder

# Checkpoint for Reprojections - if they are even necessary.
if dst_crs == 'none':
    dst_crs = 'EPSG:3857'

# Threshold the Uncertainty layer
if likelihood == True:
    with rasterio.open(input_dir / 'UNCERTAINTY_mosaic.tif') as src:
        rst = src.read(1)
        likelihood_thr = (rst > likelihood_threshold).astype(np.uint8) 
        nodata = src.nodata
        likelihood_thr[rst == nodata] = nodata
    
        out_meta = src.meta.copy()
        out_meta.update(dtype=rasterio.uint8)

        with rasterio.open(input_dir / 'Likelihood_thr.tif', "w", **out_meta) as dst:
            dst.write(likelihood_thr, 1)
    
    input_flood_delineation = input_dir / 'Likelihood_thr.tif'
    print('Likelihood Layer successfully thresholded.')
    
else:
    input_flood_delineation = input_dir / 'FLOOD_mosaic.tif'

# reproject the flood raster
with rasterio.open(input_flood_delineation) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    raster_bounds = src.bounds
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    output_filename = "flood.tif" if roi_path is None else "flood_reproj.tif"
    with rasterio.open(input_dir / output_filename, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject( 
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
    
# Crop the flood raster to the ROI
if roi_path != None:
    print('Flood Layer is clipped to AOI...')
    roi = gpd.read_file(roi_path)
    roi_reproj = roi.to_crs(dst_crs) 
    
    with rasterio.open(input_dir / 'flood_reproj.tif') as src:
        raster_bounds = src.bounds
        raster_gdf = gpd.GeoDataFrame({"geometry": [box(*raster_bounds)]}, crs = dst_crs)
        roi_crp = gpd.clip(roi_reproj, raster_gdf) # has to be cropped to the extent of the input rasters
        
        # checkpoint if roi intersects the mosaic
        if roi_crp.empty:
            raise ValueError("The AOI is empty. Please check the input AOI file.")
            
        out_image, out_transform = mask(src, roi_crp.geometry, crop = True)
        
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
            })
        with rasterio.open(input_dir / 'flood.tif', "w", **out_meta) as dst:
            dst.write(out_image)

print('Flood Layer reprojected.')

#%%
flood_dataset = rasterio.open(input_dir / 'flood.tif')
flood_array   = flood_dataset.read(1)

transform = flood_dataset.transform
flood_array_nrows, flood_array_ncol =  flood_array.shape
Dx= transform[0]
Dy= transform[4]
minX= transform[2]
maxY= transform[5]
maxX= minX + Dx*flood_array_ncol
minY= maxY + Dy*flood_array_nrows

proj = flood_dataset.crs.wkt
outputBounds=[minX, minY, maxX, maxY]

# function to reproject all input rasters other than the input flood delineation
def reproj(input_path, output_path, proj, outputBounds, continuous_input = True):
    
    raster_dataset = rasterio.open(input_path)
    input_proj       = raster_dataset.crs.wkt
    output_proj      = proj

    if continuous_input == True:
        resampling_method = 'bilinear'# other options: 'average', 'cubic', 'lanczos' ... 
    elif continuous_input == False:
        resampling_method = 'mode'   # other options: 'nearest' ...
    
    gdal.Warp(str(output_path), 
              str(input_path), 
              outputBounds=outputBounds,
              cropToCutline    = True,
              outputBoundsSRS = output_proj, 
              warpMemoryLimit= 5000,
              srcSRS= input_proj, 
              dstSRS= output_proj, 
              xRes=Dx,
              yRes=Dy, 
              resampleAlg= resampling_method,
              targetAlignedPixels = False,
              creationOptions = ["COMPRESS=ZSTD", "BIGTIFF=YES", "TILED=YES"]    # compression options: ZSTD, DEFLATE, LZW
              )
    
    #flood_dataset.close()
    raster_dataset.close()

reproj(dtm_path, input_dir / 'dtm.tif', proj, outputBounds, continuous_input = True)
reproj(input_dir / 'EXCLAYER_mosaic.tif', input_dir / 'exclusion.tif', proj, outputBounds, continuous_input = False)
reproj(input_dir / 'OBSWATER_mosaic.tif', input_dir / 'obswater.tif', proj, outputBounds, continuous_input = False)


# 6. FLEXTH
Now we are finally ready to run FLEXTH. FLEXTH takes the input rasters: 'flood.tif', 'exclusion.tif', 'dtm.tif', 'obswater.tif' in the input_dir and propagates water into the masked areas.
After it has finished, you will find the output in the folder you specified earlier. Depending on which method you used, the file name will either start with WL for Water Level or WD for Water Depth.
